# Prototype Version 1
Dec.25 2018

In [57]:
from urllib.request import urlopen
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd
import time

In [89]:
def transform_number(s):
    if s[-1] == '亿':
        return int(float(s[:-1])*1e8)
    elif s[-1] == '万':
        return int(float(s[:-1])*1e4)
    else:
        return int(s)

In [175]:
def get_read_discuss(keyword):
    link = quote('#'+keyword+'#')
    url = 'https://s.weibo.com/weibo?q='+link
    response = urlopen(url,timeout=10)
    soup = BeautifulSoup(response, 'lxml')

    text = soup.find('div', 'total')
    read = transform_number(text.find_all('span')[0].string[2:])
    discuss = transform_number(text.find_all('span')[1].string[2:])
    return read,discuss

In [176]:
from datetime import timedelta, datetime
import time
def get_time():
    if time.localtime(time.time()).tm_isdst == 0:
        dt = str(datetime.now() + timedelta(hours=13))
    else:
        dt = str(datetime.now() + timedelta(hours=12))
    year = int(dt[0:4])
    month = int(dt[5:7])
    day = int(dt[8:10])
    hour = int(dt[11:13])
    minute = int(dt[14:16])
    
    return year, month, day, hour, minute

In [179]:
def get_hot():
    year, month, day, hour, minute = get_time()
    hot_url = 'https://s.weibo.com/top/summary?cate=realtimehot'
    response = urlopen(hot_url,timeout=10)
    soup = BeautifulSoup(response, 'lxml')
    hot_list = []
    print('key_word,number, read, discuss')
    for i in soup.find_all(name='tr'):
        text = i.find('td','td-02')
        if text is not None:
            key_word = text.a.contents[0]
            link = text.a.attrs['href']
            if text.span is not None:
                number = text.span.string
                read, discuss = get_read_discuss(key_word)
                print(year, month, day, hour, minute,key_word, number, read, discuss)
                hot_list.append([year, month, day, hour, minute, key_word, number, read, discuss])
    return hot_list

In [178]:
try:
    hot_list = get_hot()
except:
    print('some error')

key_word,number, read, discuss
2018 12 29 3 59 刘德华现场中止演唱会  114990 210000000 43000
2018 12 29 3 59 周子瑜太撩了 61397 120000000 14000
some error


In [169]:
import csv
def write_csv(hot_list):
    year, month, _, _, _ = get_time()
    with open(str(year)+'-'+str(month)+'.csv', 'a') as csvFile:
        writer = csv.writer(csvFile, lineterminator='\n')
        for row in hot_list:
            writer.writerow(row)


In [170]:
write_csv(hot_list)